this is a markdown comment. the ipynb format is useful for getting individual cells to run

In [38]:
x = "Here we go!"
print(x)

Here we go!


In [39]:
import sys
print(sys.version)


3.13.5 (v3.13.5:6cb20a219a8, Jun 11 2025, 12:23:45) [Clang 16.0.0 (clang-1600.0.26.6)]


In [82]:
# Cell 1 - Imports and Setup
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import process
import pandas as pd
import time
from tqdm import tqdm

headers = {'User-Agent': 'aditya.goel.tx@gmail.com'}


In [41]:
# Cell 2 - Load Clean Fund List CSV
df_funds = pd.read_csv("Clean_Fund_List.csv")
df_funds

,fund_group,fund_id,name,ticker,fund_stkid,advt,fund_type
0,fund,04B8BC-E,American Beacon Balanced Fund,ABCCX US,135909,0,MF_O
1,fund,04B8BD-E,American Beacon Large Cap Value Fund,AADEX US,25667,0,MF_O
2,fund,04B8BK-E,RBC Enterprise Fund,TETAX US,84097,0,MF_O
3,fund,04B8BQ-E,Cohen & Steers Real Estate Securities Fund,CREFX US,210588,0,MF_O
4,fund,04B8BY-E,Voya Growth & Income Portfolio,ISVGX US,119362,0,MF_O
...,...,...,...,...,...,...,...
4837,fund,10L5FM-E,Naa World Equity Income Fund,SFGCX US,67247,0,MF_O
4838,fund,10MBT4-E,Kensington Dynamic Growth,KAGCX US,264944,0,MF_O
4839,fund,10ZJSS-E,Vontobel US Equity Fund,VNUYX US,224821,0,MF_O
4840,fund,112M3V-E,New Age Alpha SMid Cap Value Fund,SEVSX US,67504,0,MF_O


In [77]:


def get_cik_from_mutual_fund(fund_name):
    # Format the URL with the fund name
    base_url = "https://www.sec.gov/cgi-bin/series"
    params = {
        "company": fund_name,
        "sc": "companyseries",
        "type": "N-PX"
    }

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code != 200:
        print("Error fetching data from SEC.")
        return None

    # Parse HTML response
    soup = BeautifulSoup(response.text, "html.parser")

    # Look for links that contain 'CIK=' in href
    links = soup.find_all('a', href=True)

    for link in links:
        href = link['href']
        if 'CIK=' in href:
            # Extract CIK value
            cik_value = href.split("CIK=")[1].split("&")[0]
            return cik_value

    #print("CIK not found in the page.")
    return None

In [72]:
# Try the first fund in your list
test_name = df_funds.iloc[10]['name'] 
print("Testing fund:", test_name)

# Try retrieving CIK
cik = get_cik_from_mutual_fund(test_name)
print("CIK found:", cik)


Testing fund: Columbia Acorn Fund
CIK found: 0000002110


In [86]:


# Store results here
results = []
save_every = 25  # Save partial progress every 25 funds

# Use tqdm for progress bar
for idx, fund in tqdm(enumerate(df_funds['name']), total=len(df_funds)):
    try:
        cik = get_cik_from_mutual_fund(fund)
        results.append({'Fund_Name': fund, 'CIK': cik})
        print(f"[{idx}] {fund} → {cik}")
    except Exception as e:
        print(f"❌ Error for {fund}: {e}")
        results.append({'Fund_Name': fund, 'CIK': None})
    
    time.sleep(0.1)  # Rate limiting

    # Save progress every N
    if idx % save_every == 0 and idx != 0:
        pd.DataFrame(results).to_csv('Fund_CIK_Mappings_partial.csv', index=False)

# Final save
df_cik_map = pd.DataFrame(results)
df_cik_map.to_csv('Fund_CIK_Mappings_FINAL.csv', index=False)

print("\n✅ All done! Final CSV saved.")


  0%|          | 1/4842 [00:00<1:16:53,  1.05it/s]

[0] American Beacon Balanced Fund → 0000809593


  0%|          | 2/4842 [00:01<54:12,  1.49it/s]  

[1] American Beacon Large Cap Value Fund → 0000809593


  0%|          | 3/4842 [00:01<48:40,  1.66it/s]

...

[4839] Vontobel US Equity Fund → None


100%|█████████▉| 4841/4842 [27:21<00:00,  2.71it/s]

[4840] New Age Alpha SMid Cap Value Fund → 0002013853


100%|██████████| 4842/4842 [27:22<00:00,  2.95it/s]

[4841] SEGALL BRYANT & HAMILL TRUST - Barrett Opportunity Fund → None

✅ All done! Final CSV saved.
